## Importar librerias

In [1]:
# Manejo de datos
import numpy as np
import pandas as pd
import os
from datetime import datetime

# Gráficos (Incluidos únicamente en el archivo .ipybn)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Preprocesado y modelado
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import skforecast
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import grid_search_forecaster

#Cargar y guardar modelos (Opcional)
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# Configuración warnings (Quitar en caso de errores desconocidos)
import warnings
warnings.filterwarnings('ignore')

# Versión de paquetes usados
color = '\033[1m\033[38;5;208m'
print(f"{color}Versión de las librerias utilizadas:")
print(f"{color}- Version skforecast: {skforecast.__version__}")
print(f"{color}- Version scikit-learn: {sklearn.__version__}")
print(f"{color}- Version pandas: {pd.__version__}")
print(f"{color}- Version numpy: {np.__version__}")

Versión de las librerias utilizadas:
- Version skforecast: 0.14.0
- Version scikit-learn: 1.5.0
- Version pandas: 2.2.3
- Version numpy: 1.26.4


## Lectura del dataset

In [ ]:
# Cargar el archivo Excel
df = pd.read_excel("accesos.xlsx")

# Asegurar que la fecha esté en formato datetime
df['Fecha Completa'] = pd.to_datetime(df['Fecha Completa'])

# Crear variables temporales relevantes
df['Día'] = df['Fecha Completa'].dt.day
df['Mes'] = df['Fecha Completa'].dt.month
df['Año'] = df['Fecha Completa'].dt.year

# Codificar Jornada, Pregrado y Postgrado
df['Jornada'] = df['Jornada'].astype('category').cat.codes
df['Pregrado'] = df['Pregrado'].map({'SI': 1, 'NO': 0})
df['Postgrado'] = df['Postgrado'].map({'SI': 1, 'NO': 0})

# Crear una clave de fecha sin hora
df['Fecha'] = df['Fecha Completa'].dt.date

# Agrupar y contar accesos
daily_counts = df.groupby(['Fecha', 'Día', 'Mes', 'Año']).agg({
    'Jornada': 'count',
    'Pregrado': 'count',
    'Postgrado': 'count',
    'Fecha Completa': 'count'  # esto cuenta los accesos
}).rename(columns={'Fecha Completa': 'Accesos'}).reset_index()

In [3]:
daily_counts.head(10)

,Fecha,Día,Mes,Año,Jornada,Pregrado,Postgrado,Accesos
0,2023-12-14,14,12,2023,1.0,0.697796,0.017838,953
1,2023-12-15,15,12,2023,1.0,0.719490,0.009107,549
2,2023-12-16,16,12,2023,0.0,0.676056,0.028169,71
3,2023-12-18,18,12,2023,1.0,0.676590,0.020298,739
4,2023-12-19,19,12,2023,1.0,0.637076,0.020888,383
5,2023-12-20,20,12,2023,1.0,0.556485,0.025105,239
6,2023-12-21,21,12,2023,1.0,0.582938,0.047393,211
7,2023-12-22,22,12,2023,1.0,0.393443,0.032787,122
8,2024-01-02,2,1,2024,1.0,0.512821,0.025641,156
9,2024-01-03,3,1,2024,1.0,0.699153,0.008475,472


In [4]:
daily_counts.tail(10)

,Fecha,Día,Mes,Año,Jornada,Pregrado,Postgrado,Accesos
324,2025-04-24,24,4,2025,1.0,0.971698,0.004717,2544
325,2025-04-25,25,4,2025,1.0,0.971976,0.006637,1356
326,2025-04-26,26,4,2025,0.0,0.922053,0.013308,526
327,2025-04-27,27,4,2025,0.0,1.000000,0.000000,4
328,2025-04-28,28,4,2025,1.0,0.973333,0.007917,2400
329,2025-04-29,29,4,2025,1.0,0.971963,0.007009,2568
330,2025-04-30,30,4,2025,1.0,0.975367,0.007625,1705
331,2025-05-05,5,5,2025,1.0,0.977455,0.005455,2750
332,2025-05-06,6,5,2025,1.0,0.968917,0.002680,1866
333,2025-05-07,7,5,2025,1.0,0.980923,0.002218,2254
